example: suppose a group of workers needs to perform a set of tasks, and for each worker and task, there is a cost for assigning the worker to the task. The problem is to assign each worker to at most one task, with no two workers performing the same task, while minimizing the total cost.

один работник выполняет 1 задачу, 1 задача выполняется 1 работником

In the example there are five workers (numbered 0-4) and four tasks (numbered 0-3)<br>
0	1	2	3<br>
0	90	80	75	70<br>
1	35	85	55	65<br>
2	125	95	90	95<br>
3	45	110	95	115<br>
4	50	100	90	100<br>

In [3]:
from ortools.linear_solver import pywraplp

# Data
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [50, 100, 90, 100],
]
num_workers = len(costs)
num_tasks = len(costs[0])

# Solver
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')


# Variables
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, '')

# Constraints
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

# Objective
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

# Solve
status = solver.Solve()

# Print solution.
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Total cost = ', solver.Objective().Value(), '\n')
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('Worker %d assigned to task %d.  Cost = %d' %
                      (i, j, costs[i][j]))

Total cost =  265.0 

Worker 0 assigned to task 3.  Cost = 70
Worker 1 assigned to task 2.  Cost = 55
Worker 2 assigned to task 1.  Cost = 95
Worker 3 assigned to task 0.  Cost = 45


# The following sections describe how to solve the problem using the CP-SAT solver.

In [4]:
from ortools.sat.python import cp_model

# Data
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [50, 100, 90, 100],
]
num_workers = len(costs)
num_tasks = len(costs[0])

# Model
model = cp_model.CpModel()

# Variables
x = []
for i in range(num_workers):
    t = []
    for j in range(num_tasks):
        t.append(model.NewBoolVar('x[%i,%i]' % (i, j)))
    x.append(t)

# Constraints
# Each worker is assigned to at most one task.
for i in range(num_workers):
    model.Add(sum(x[i][j] for j in range(num_tasks)) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    model.Add(sum(x[i][j] for i in range(num_workers)) == 1)

# Objective
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i][j])
model.Minimize(sum(objective_terms))

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Total cost = %i' % solver.ObjectiveValue())
    print()
    for i in range(num_workers):
        for j in range(num_tasks):
            if solver.BooleanValue(x[i][j]):
                print('Worker ', i, ' assigned to task ', j, '  Cost = ',
                      costs[i][j])
else:
    print('No solution found.')

Total cost = 265

Worker  0  assigned to task  3   Cost =  70
Worker  1  assigned to task  2   Cost =  55
Worker  2  assigned to task  1   Cost =  95
Worker  3  assigned to task  0   Cost =  45


# Assignment with Teams of Workers

six workers are divided into two teams, and each team can perform at most two tasks

In [11]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('SolveAssignmentProblemMIP',
                       pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

cost = [[90, 76, 75, 70],
      [35, 85, 55, 65],
      [125, 95, 90, 105],
      [45, 110, 95, 115],
      [60, 105, 80, 75],
      [45, 65, 110, 95]]

team1 = [0, 2, 4]
team2 = [1, 3, 5]
team_max = 2

num_workers = len(cost)
num_tasks = len(cost[1])
x = {}

for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))

# Objective
solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                              for j in range(num_tasks)]))

# Constraints

# Each worker is assigned to at most 1 task.

for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.

for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

# Each team takes on two tasks.

solver.Add(solver.Sum([x[i, j] for i in team1 for j in range(num_tasks)]) <= team_max)
solver.Add(solver.Sum([x[i, j] for i in team2 for j in range(num_tasks)]) <= team_max)
sol = solver.Solve()

print('Total cost = ', solver.Objective().Value())
print()
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
            print('Worker %d assigned to task %d.  Cost = %d' % (i,j,cost[i][j]))

print()
print("Time = ", solver.WallTime(), " milliseconds")

Total cost =  250.0

Worker 0 assigned to task 2.  Cost = 75
Worker 1 assigned to task 0.  Cost = 35
Worker 4 assigned to task 3.  Cost = 75
Worker 5 assigned to task 1.  Cost = 65

Time =  8  milliseconds


# Assignment with Task Sizes

each task has a size, which represents how much time or effort the task requires. The total size of the tasks performed by each worker has a fixed bound

CP-SAT

In [12]:
from __future__ import print_function
from ortools.sat.python import cp_model
import time
import numpy as np

model = cp_model.CpModel()

start = time.time()
#As in previous examples, the cost matrix gives the cost for worker i to perform task j. 
#The sizes vector gives the size of each task. 
#total_size_max is the upper bound on the total size of the tasks performed by any single worker.
cost = [[90, 76, 75, 70, 50, 74, 12, 68],
      [35, 85, 55, 65, 48, 101, 70, 83],
      [125, 95, 90, 105, 59, 120, 36, 73],
      [45, 110, 95, 115, 104, 83, 37, 71],
      [60, 105, 80, 75, 59, 62, 93, 88],
      [45, 65, 110, 95, 47, 31, 81, 34],
      [38, 51, 107, 41, 69, 99, 115, 48],
      [47, 85, 57, 71, 92, 77, 109, 36],
      [39, 63, 97, 49, 118, 56, 92, 61],
      [47, 101, 71, 60, 88, 109, 52, 90]]

sizes = [10, 7, 3, 12, 15, 4, 11, 5]
total_size_max = 15
num_workers = len(cost)
num_tasks = len(cost[1])
# Variables
x = []
for i in range(num_workers):
    t = []
    for j in range(num_tasks):
        t.append(model.NewIntVar(0, 1, "x[%i,%i]" % (i, j)))
    x.append(t)
x_array = [x[i][j] for i in range(num_workers) for j in range(num_tasks)]

# Constraints

# Each task is assigned to at least one worker.
[model.Add(sum(x[i][j] for i in range(num_workers)) >= 1) for j in range(num_tasks)]

# Total size of tasks for each worker is at most total_size_max.

[model.Add(sum(sizes[j] * x[i][j] for j in range(num_tasks)) <= total_size_max) for i in range(num_workers)]
model.Minimize(sum([np.dot(x_row, cost_row) for (x_row, cost_row) in zip(x, cost)]))
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Minimum cost = %i' % solver.ObjectiveValue())
    print()

    for i in range(num_workers):
        for j in range(num_tasks):
            if solver.Value(x[i][j]) == 1:
                print('Worker ', i, ' assigned to task ', j, '  Cost = ', cost[i][j])
    print()
    end = time.time()
    print("Time = ", round(end - start, 4), "seconds")

Minimum cost = 326

Worker  0  assigned to task  6   Cost =  12
Worker  1  assigned to task  0   Cost =  35
Worker  1  assigned to task  2   Cost =  55
Worker  4  assigned to task  4   Cost =  59
Worker  5  assigned to task  5   Cost =  31
Worker  5  assigned to task  7   Cost =  34
Worker  6  assigned to task  1   Cost =  51
Worker  8  assigned to task  3   Cost =  49

Time =  2.5561 seconds


MIP program

In [13]:
import time
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('SolveAssignmentProblem',
                       pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start = time.time()
cost = [[90, 76, 75, 70, 50, 74, 12, 68],
      [35, 85, 55, 65, 48, 101, 70, 83],
      [125, 95, 90, 105, 59, 120, 36, 73],
      [45, 110, 95, 115, 104, 83, 37, 71],
      [60, 105, 80, 75, 59, 62, 93, 88],
      [45, 65, 110, 95, 47, 31, 81, 34],
      [38, 51, 107, 41, 69, 99, 115, 48],
      [47, 85, 57, 71, 92, 77, 109, 36],
      [39, 63, 97, 49, 118, 56, 92, 61],
      [47, 101, 71, 60, 88, 109, 52, 90]]

task_sizes = [10, 7, 3, 12, 15, 4, 11, 5]

# Maximum total of task sizes for any worker
total_size_max = 15
num_workers = len(cost)
num_tasks = len(cost[1])
# Variables
x = {}

for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

# Constraints
# The total size of the tasks each worker takes on is at most total_size_max.
for i in range(num_workers):
    solver.Add(solver.Sum([task_sizes[j] * x[i, j] for j in range(num_tasks)]) <= total_size_max)

# Each task is assigned to at least one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) >= 1)

solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                              for j in range(num_tasks)]))
sol = solver.Solve()

print('Minimum cost = ', solver.Objective().Value())
print()
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
            print('Worker', i,' assigned to task', j, '  Cost = ', cost[i][j])
print()
end = time.time()
print("Time = ", round(end - start, 4), "seconds")

Minimum cost =  326.0

Worker 0  assigned to task 6   Cost =  12
Worker 1  assigned to task 0   Cost =  35
Worker 1  assigned to task 2   Cost =  55
Worker 4  assigned to task 4   Cost =  59
Worker 5  assigned to task 5   Cost =  31
Worker 5  assigned to task 7   Cost =  34
Worker 6  assigned to task 1   Cost =  51
Worker 8  assigned to task 3   Cost =  49

Time =  0.0177 seconds


# Assignment with Allowed Groups

only certain allowed groups of workers can be assigned to the tasks. In the example there are twelve workers, numbered 0 - 11. The allowed groups are combinations of the following pairs of workers

Можно взять любую пару из группы

MIP

In [16]:
from ortools.linear_solver import pywraplp
import time

start = time.time()
cost = [[90, 76, 75, 70, 50, 74],
      [35, 85, 55, 65, 48, 101],
      [125, 95, 90, 105, 59, 120],
      [45, 110, 95, 115, 104, 83],
      [60, 105, 80, 75, 59, 62],
      [45, 65, 110, 95, 47, 31],
      [38, 51, 107, 41, 69, 99],
      [47, 85, 57, 71, 92, 77],
      [39, 63, 97, 49, 118, 56],
      [47, 101, 71, 60, 88, 109],
      [17, 39, 103, 64, 61, 92],
      [101, 45, 83, 59, 92, 27]]

num_tasks = len(cost[1])
# Allowed groups of workers:
group1 =  [[2, 3],       # Subgroups of workers 0 - 3
         [1, 3],
         [1, 2],
         [0, 1],
         [0, 2]]

group2 =  [[6, 7],       # Subgroups of workers 4 - 7
         [5, 7],
         [5, 6],
         [4, 5],
         [4, 7]]

group3 =  [[10, 11],     # Subgroups of workers 8 - 11
         [9, 11],
         [9, 10],
         [8, 10],
         [8, 11]]

def assignment(cost, group):
    # Solve assignment problem for given group of workers.
    num_tasks = len(cost[1])
    # Clear values in x
    solver = None
    # Instantiate a mixed-integer solver
    solver = pywraplp.Solver('AssignmentProblemGroups',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    x = None
    x = {}

    for i in group:
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

    # Each worker is assigned to exactly one task.

    for i in group:
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to at least one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in group]) >= 1)
    # Objective
    solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in group
                                                  for j in range(num_tasks)]))
    solver.Solve()
    res = [solver, x]
    return res

allowed_groups = []

for i in range(len(group1)):
    for j in range(len(group2)):
        for k in range(len(group3)):
            allowed_groups.append(group1[i] + group2[j] + group3[k])
min_val = 1e6
total_time = 0

for i in range(len(allowed_groups)):
    group = allowed_groups[i]
    res = assignment(cost, group)
    solver_tmp = res[0]
    x_tmp = res[1]
    total_time = total_time + solver_tmp.WallTime()

    if solver_tmp.Objective().Value() < min_val:
        min_val = solver_tmp.Objective().Value()
        min_index = i
        min_solver = solver_tmp
        min_x = x_tmp
        min_group = group

print('Minimum cost = ', min_val)
print()
for i in min_group:
    for j in range(num_tasks):
        if min_x[i, j].solution_value() > 0:
            print('Worker', i,' assigned to task', j, '  Cost = ', cost[i][j])
print()
end = time.time()
print("Time = ", round(end - start, 4), "seconds")

Minimum cost =  239.0

Worker 0  assigned to task 4   Cost =  50
Worker 1  assigned to task 2   Cost =  55
Worker 5  assigned to task 5   Cost =  31
Worker 6  assigned to task 3   Cost =  41
Worker 10  assigned to task 0   Cost =  17
Worker 11  assigned to task 1   Cost =  45

Time =  0.3952 seconds


CP-SAT program

In [17]:
from ortools.sat.python import cp_model
import time
import numpy as np

model = cp_model.CpModel()
start = time.time()
cost = [[90, 76, 75, 70, 50, 74],
      [35, 85, 55, 65, 48, 101],
      [125, 95, 90, 105, 59, 120],
      [45, 110, 95, 115, 104, 83],
      [60, 105, 80, 75, 59, 62],
      [45, 65, 110, 95, 47, 31],
      [38, 51, 107, 41, 69, 99],
      [47, 85, 57, 71, 92, 77],
      [39, 63, 97, 49, 118, 56],
      [47, 101, 71, 60, 88, 109],
      [17, 39, 103, 64, 61, 92],
      [101, 45, 83, 59, 92, 27]]
num_workers = len(cost)
num_tasks = len(cost[1])
group1 =  [[0, 0, 1, 1],      # Workers 2, 3
         [0, 1, 0, 1],      # Workers 1, 3
         [0, 1, 1, 0],      # Workers 1, 2
         [1, 1, 0, 0],      # Workers 0, 1
         [1, 0, 1, 0]]      # Workers 0, 2

group2 =  [[0, 0, 1, 1],      # Workers 6, 7
         [0, 1, 0, 1],      # Workers 5, 7
         [0, 1, 1, 0],      # Workers 5, 6
         [1, 1, 0, 0],      # Workers 4, 5
         [1, 0, 0, 1]]      # Workers 4, 7

group3 =  [[0, 0, 1, 1],      # Workers 10, 11
         [0, 1, 0, 1],      # Workers 9, 11
         [0, 1, 1, 0],      # Workers 9, 10
         [1, 0, 1, 0],      # Workers 8, 10
         [1, 0, 0, 1]]      # Workers 8, 11

# Declare the variables.
x = []
for i in range(num_workers):
    t = []
    for j in range(num_tasks):
        t.append(model.NewIntVar(0, 1, "x[%i,%i]" % (i, j)))
    x.append(t)
x_array = [x[i][j] for i in range(num_workers) for j in range(num_tasks)]
# Constraints

# Each task is assigned to at least one worker.
[model.Add(sum(x[i][j] for i in range(num_workers)) == 1) for j in range(num_tasks)]

# Each worker is assigned to at most one task.
[model.Add(sum(x[i][j] for j in range(num_tasks)) <= 1) for i in range(num_workers)]

# Create variables for each worker, indicating whether they work on some task.
work = []
for i in range(num_workers):
    work.append(model.NewIntVar(0, 1, "work[%i]" % i))

for i in range(num_workers):
    for j in range(num_tasks):
        model.Add(work[i] == sum(x[i][j] for j in range(num_tasks)))

# Define the allowed groups of worders
model.AddAllowedAssignments([work[0], work[1], work[2], work[3]], group1)
model.AddAllowedAssignments([work[4], work[5], work[6], work[7]], group2)
model.AddAllowedAssignments([work[8], work[9], work[10], work[11]], group3)
model.Minimize(sum([np.dot(x_row, cost_row) for (x_row, cost_row) in zip(x, cost)]))
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Minimum cost = %i' % solver.ObjectiveValue())
    print()
    for i in range(num_workers):
        for j in range(num_tasks):
            if solver.Value(x[i][j]) == 1:
                print('Worker ', i, ' assigned to task ', j, '  Cost = ', cost[i][j])
    print()
    end = time.time()
    print("Time = ", round(end - start, 4), "seconds")

Minimum cost = 239

Worker  0  assigned to task  4   Cost =  50
Worker  1  assigned to task  2   Cost =  55
Worker  5  assigned to task  5   Cost =  31
Worker  6  assigned to task  3   Cost =  41
Worker  10  assigned to task  0   Cost =  17
Worker  11  assigned to task  1   Cost =  45

Time =  0.1229 seconds


# Linear Assignment Solver

Подходит для простых assignment problems

4 сотрудник и 4 задачи

In [18]:
from __future__ import print_function
from ortools.graph import pywrapgraph
import time

def create_data_array():
    cost = [[90, 76, 75, 70],
            [35, 85, 55, 65],
            [125, 95, 90, 105],
            [45, 110, 95, 115]]
    return cost


cost = create_data_array()
rows = len(cost)
cols = len(cost[0])

assignment = pywrapgraph.LinearSumAssignment()
for worker in range(rows):
    for task in range(cols):
        if cost[worker][task]:
            assignment.AddArcWithCost(worker, task, cost[worker][task])
solve_status = assignment.Solve()
if solve_status == assignment.OPTIMAL:
    print('Total cost = ', assignment.OptimalCost())
    print()
    for i in range(0, assignment.NumNodes()):
        print('Worker %d assigned to task %d.  Cost = %d' % (
          i,
          assignment.RightMate(i),
          assignment.AssignmentCost(i)))
elif solve_status == assignment.INFEASIBLE:
    print('No assignment is possible.')
elif solve_status == assignment.POSSIBLE_OVERFLOW:
    print('Some input costs are too large and may cause an integer overflow.')

Total cost =  265

Worker 0 assigned to task 3.  Cost = 70
Worker 1 assigned to task 2.  Cost = 55
Worker 2 assigned to task 1.  Cost = 95
Worker 3 assigned to task 0.  Cost = 45
